In [1]:
import json
import pandas as pd

In [31]:
from hu_prompts import hu_PROMPTS

In [ ]:
# df = pd.read_parquet('/mnt/petrelfs/hujucheng/huLLM/data/pretrain/train-00002-of-00003.parquet')
# df.to_json('/mnt/petrelfs/hujucheng/huLLM/data/pretrain/type1_3.csv')


In [25]:
def transform_and_save_json(input_file, output_file):
    # Read the input JSON from a file
    with open(input_file, "r") as infile:
        input_json = json.load(infile)
    
    output_json = []

    for item in input_json:
        question = item["instruction"]
        options = item["input"]
        rationale = item["output"]

        
        # Construct the f-string
        f_string = f"{question}\n{options}\n\n"
       
        # Construct the message template
        message = {
            "messages": [
                {
                    "role": "user",
                    "content": f_string
                },
                {
                    "role": "assistant",
                    "content": rationale
                }
            ]
        }
        output_json.append(message)
    
    # Write the transformed JSON to the output file
    with open(output_file, "w") as outfile:
        json.dump(output_json, outfile, indent=4,ensure_ascii=False)

    print(f"Transformed JSON has been saved to {output_file}")

# Example usage
input_file_path = "/mnt/petrelfs/hujucheng/huLLM/data/tmp/tagged-pixiv-novel.json"  # Replace with your input file path
output_file_path = "/mnt/petrelfs/hujucheng/huLLM/data/SFT/write_by_tag_3"  # Replace with your output file path

transform_and_save_json(input_file_path, output_file_path)


Transformed JSON has been saved to /mnt/petrelfs/hujucheng/huLLM/data/SFT/write_by_tag_3


In [ ]:
import os
import json

def txts_to_json(directory_path, output_file):
    data = []

    # Iterate over all files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                
                # Split content into paragraphs
                paragraphs = [p.strip() for p in content.split('\n\n') if p.strip()]
                crt_filename = filename[:-4]
                
                # Initialize a list to hold the current combined paragraph
                combined_paragraphs = []
                current_word_count = 0
                
                # Add paragraphs to data list with filename as a key
                for paragraph in paragraphs:
                    paragraph_words = paragraph.split()
                    if current_word_count + len(paragraph_words) < 100:
                        # If adding this paragraph doesn't exceed 100 words, add it to the current combined paragraph
                        combined_paragraphs.append(paragraph)
                        current_word_count += len(paragraph_words)
                    else:
                        # If the current combined paragraph has less than 100 words, create a new JSON object
                        if combined_paragraphs:
                            message = {
                                "messages": [
                                    {
                                        "role": "user",
                                        "content": f"{hu_PROMPTS[1]}\n题目如下：{crt_filename}"
                                    },
                                    {
                                        "role": "assistant",
                                        "content": ' '.join([p for p in combined_paragraphs])
                                    }
                                ]
                            }
                            data.append(message)
                        # Reset the combined paragraph and word count
                        combined_paragraphs = [paragraph]
                        current_word_count = len(paragraph_words)
                
                # Don't forget to add the last combined paragraph if it has less than 100 words
                if combined_paragraphs:
                    message = {
                        "messages": [
                            {
                                "role": "user",
                                "content": f"{hu_PROMPTS[1]}\n题目如下：{crt_filename}"
                            },
                            {
                                "role": "assistant",
                                "content": ' '.join([p for p in combined_paragraphs])
                            }
                        ]
                    }
                    data.append(message)

    # Write the data to a JSON file
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

# Example usage
directory_path = '/mnt/petrelfs/hujucheng/huLLM/data/pretrain/text'
output_file = '/mnt/petrelfs/hujucheng/huLLM/data/SFT/write_by_title.json'
txts_to_json(directory_path, output_file)

In [32]:
import os
import json

def txts_to_json(directory_path, output_file):
    data = []

    # Walk through all directories and files in the directory tree
    for dirpath, dirnames, filenames in os.walk(directory_path):
        for filename in filenames:
            if filename.endswith('.txt'):
                file_path = os.path.join(dirpath, filename)
                
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    
                    # Split content into paragraphs
                    paragraphs = [p.strip() for p in content.split('\n\n') if p.strip()]
                    crt_filename = filename[:-4]
                    
                    # Initialize a list to hold the current combined paragraph
                    combined_paragraphs = []
                    current_word_count = 0
                    
                    # Add paragraphs to data list with filename as a key
                    for paragraph in paragraphs:
                        paragraph_words = paragraph.split()
                        if current_word_count + len(paragraph_words) < 100:
                            # If adding this paragraph doesn't exceed 100 words, add it to the current combined paragraph
                            combined_paragraphs.append(paragraph)
                            current_word_count += len(paragraph_words)
                        else:
                            # If the current combined paragraph has less than 100 words, create a new JSON object
                            if combined_paragraphs:
                                message = {
                                    "messages": [
                                        {
                                            "role": "user",
                                            "content": f"{hu_PROMPTS[1]}\n题目如下：{crt_filename}"
                                        },
                                        {
                                            "role": "assistant",
                                            "content": ' '.join([p for p in combined_paragraphs])
                                        }
                                    ]
                                }
                                data.append(message)
                            # Reset the combined paragraph and word count
                            combined_paragraphs = [paragraph]
                            current_word_count = len(paragraph_words)
                    
                    # Don't forget to add the last combined paragraph if it has less than 100 words
                    if combined_paragraphs:
                        message = {
                            "messages": [
                                {
                                    "role": "user",
                                    "content": f"{hu_PROMPTS[1]}\n题目如下：{crt_filename}"
                                },
                                {
                                    "role": "assistant",
                                    "content": ' '.join([p for p in combined_paragraphs])
                                }
                            ]
                        }
                        data.append(message)

    # Write the data to a JSON file
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

# Example usage
directory_path = '/mnt/petrelfs/hujucheng/huLLM/data/tmp'
output_file = '/mnt/petrelfs/hujucheng/huLLM/data/SFT/write_by_title_2.json'
txts_to_json(directory_path, output_file)

In [ ]:
lines = []
with open("/mnt/petrelfs/hujucheng/huLLM/data/pretrain/SexNovel.jsonl","r") as check:
    for line in check:
        lines.append(line)


In [ ]:
def transform_and_save_json(lines, output_file):

    output_json = []

    for item in lines:
        text = json.loads(item)

        rationale = text["text"]

        
        # Construct the f-string
        f_string = f"发挥创造力写一篇包含色情NSFW内容的短篇故事"
       
        # Construct the message template
        message = {
            "messages": [
                {
                    "role": "user",
                    "content": f_string
                },
                {
                    "role": "assistant",
                    "content": rationale
                }
            ]
        }
        output_json.append(message)
    
    # Write the transformed JSON to the output file
    with open(output_file, "w") as outfile:
        json.dump(output_json, outfile, indent=4,ensure_ascii=False)

    print(f"Transformed JSON has been saved to {output_file}")

# Example usage
output_file_path = "/mnt/petrelfs/hujucheng/huLLM/data/SFT/creativity_writing_1.json"  # Replace with your output file path

transform_and_save_json(lines,output_file_path)


Transformed JSON has been saved to /mnt/petrelfs/hujucheng/huLLM/data/SFT/creativity_writing_1.json


In [29]:
with open("/mnt/petrelfs/hujucheng/huLLM/data/pretrain/test.json","r") as check:
    a = json.load(check)

In [ ]:
a

In [34]:
def transform_and_save_json(input_file, output_file):
    # Read the input JSON from a file
    with open(input_file, "r") as infile:
        input_json = json.load(infile)
    
    output_json = []

    for item in input_json:
        tags = item["tags"]
        content = item["content"]
       
        # Construct the f-string
        f_string = f"{hu_PROMPTS[0]}\n{tags}"
       
        # Construct the message template
        message = {
            "messages": [
                {
                    "role": "user",
                    "content": f_string
                },
                {
                    "role": "assistant",
                    "content": content
                }
            ]
        }
        output_json.append(message)
    
    # Write the transformed JSON to the output file
    with open(output_file, "w") as outfile:
        json.dump(output_json, outfile, indent=4,ensure_ascii=False)

    print(f"Transformed JSON has been saved to {output_file}")

# Example usage
input_file_path = "/mnt/petrelfs/hujucheng/huLLM/data/pretrain/train.json"  # Replace with your input file path
output_file_path = "/mnt/petrelfs/hujucheng/huLLM/data/SFT/write_by_tag_2.json"  # Replace with your output file path

transform_and_save_json(input_file_path, output_file_path)


Transformed JSON has been saved to /mnt/petrelfs/hujucheng/huLLM/data/SFT/write_by_tag_2.json


In [35]:
with open("/mnt/petrelfs/hujucheng/huLLM/data/SFT/write_by_tag_2.json","r") as check:
    a = json.load(check)

In [20]:
df = pd.read_parquet('/mnt/petrelfs/hujucheng/huLLM/data/pretrain/train-00001-of-00003.parquet')


In [21]:
df = df[["content","summary3"]]

In [22]:
substring = "根据故事的梗概创作一个包含色情和NSFW内容的故事片段："


df['summary3'] = df['summary3'].apply(lambda x: substring + str(x))
messages_list = []
for index, row in df.iterrows():
    message = {
        "messages": [
            {
                "role": "user",
                "content": row['summary3']
            },
            {
                "role": "assistant",
                "content": row['content']
            }
        ]
    }
    messages_list.append(message)

# Convert the list of dictionaries to a JSON string
json_str = pd.json_normalize(messages_list).to_json(orient='records')



with open('/mnt/petrelfs/hujucheng/huLLM/data/SFT/write_by_summary_3.json', 'w', encoding='utf-8') as f:
    f.write(json_str)

In [33]:
with open("/mnt/petrelfs/hujucheng/huLLM/data/SFT/write_by_title_2.json","r") as check:
    a = json.load(check)